In [1]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import numpy as np

In [2]:
df_datos_demo = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_demograficos.csv")
df_datos_generacion = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_generacion.csv")
df_datos_PIB = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\datos_PIB.csv")
df_anio = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_anio.csv")
df_ccaa= pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_ccaa.csv")
df_demanda = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_demanda.csv")
df_edades = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_edades.csv")
df_nac_extran = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_nac_extran.csv")
df_provincias = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_provincias.csv")
df_sexos = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_sexos.csv")
df_tipo_generacion = pd.read_csv("C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos\\df_tipo_generacion.csv")

In [3]:
df_demanda.dtypes

Energía (MWh)            float64
Porcentage_Generacion      int64
COD-CCAA                   int64
Año                        int64
Mes                        int64
dtype: object

In [4]:
df_datos_demo = df_datos_demo.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_datos_generacion = df_datos_generacion.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_datos_PIB = df_datos_PIB.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')
df_demanda = df_demanda.merge(df_anio, how='inner', left_on= 'Año', right_on= 'Año').drop(columns='Año')

In [5]:
try:
    conexion = psycopg2.connect(
    database = 'ETL',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [6]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_CCAA = """ 
    CREATE TABLE IF NOT EXISTS ccaa (
    COD_CCAA int PRIMARY KEY,
    CCAA varchar(100));

"""

cursor.execute(query_creacion_CCAA)
conexion.commit()

In [7]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_type_gen = """ 
    CREATE TABLE IF NOT EXISTS tipo_generacion (
    id_tipo_generacion int PRIMARY KEY,
    tipo_generacion varchar(100));

"""

cursor.execute(query_creacion_type_gen)
conexion.commit()

In [8]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_sexo = """ 
    CREATE TABLE IF NOT EXISTS sexo (
    id_sexo int PRIMARY KEY,
    sexo varchar(100));

"""

cursor.execute(query_creacion_sexo)
conexion.commit()

In [9]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_grupo_edades = """ 
    CREATE TABLE IF NOT EXISTS edad (
    id_grupo_edad int PRIMARY KEY,
    grupo_edad varchar(100));

"""

cursor.execute(query_creacion_grupo_edades)
conexion.commit()

In [10]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_grupo_nac_extr = """ 
    CREATE TABLE IF NOT EXISTS nacionales_extranjeros (
    id_categoria int PRIMARY KEY,
    categoria varchar(100));

"""

cursor.execute(query_creacion_grupo_nac_extr)
conexion.commit()

In [11]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_provincias = """ 
    CREATE TABLE IF NOT EXISTS provincias (
    id_provincia int PRIMARY KEY,
    provincia varchar(100));

"""

cursor.execute(query_creacion_provincias)
conexion.commit()

In [12]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_anio = """ 
    CREATE TABLE IF NOT EXISTS anio (
    id_anio int PRIMARY KEY,
    anio int);

"""

cursor.execute(query_creacion_anio)
conexion.commit()

In [13]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_d_demo = """ 
    CREATE TABLE IF NOT EXISTS datos_demograficos (
    total int,
    id_provincia int, 
    id_grupo_edad int,
    id_categoria int,
    id_sexo int, 
    id_anio int,     
    CONSTRAINT fk_provincias FOREIGN KEY (id_provincia)
        REFERENCES provincias(id_provincia),
    CONSTRAINT fk_edad FOREIGN KEY (id_grupo_edad)
        REFERENCES edad(id_grupo_edad),
    CONSTRAINT fk_nacionales_extranjeros FOREIGN KEY (id_categoria)
        REFERENCES nacionales_extranjeros(id_categoria),
    CONSTRAINT fk_sexo FOREIGN KEY (id_sexo)
        REFERENCES sexo(id_sexo),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""
# recuerda que cambiaste el nombre del id_categoria por nacionales_extranjeros
cursor.execute(query_creacion_d_demo)
conexion.commit()

In [14]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_pib = """ 
    CREATE TABLE IF NOT EXISTS pib (
    ramas_actividad varchar(300),
    total int, 
    id_provincia int,
    id_anio int,     
    CONSTRAINT fk_provincias FOREIGN KEY (id_provincia)
        REFERENCES provincias(id_provincia),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_pib)
conexion.commit()

In [15]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_generacion = """ 
    CREATE TABLE IF NOT EXISTS generacion (
    energia_mwh float,
    porcentage_generacion float, 
    COD_CCAA int,
    mes int,
    id_tipo_generacion int, 
    id_anio int,     
    CONSTRAINT fk_ccaa FOREIGN KEY (COD_CCAA)
        REFERENCES ccaa(COD_CCAA),
    CONSTRAINT fk_tipo_generacion FOREIGN KEY (id_tipo_generacion)
        REFERENCES tipo_generacion(id_tipo_generacion),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_generacion)
conexion.commit()

In [16]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_demanda = """ 
    CREATE TABLE IF NOT EXISTS demanda (
    energia_mwh_ float,
    porcentage_generacion int, 
    COD_CCAA int,
    mes int,
    id_anio int,     
    CONSTRAINT fk_ccaa FOREIGN KEY (COD_CCAA)
        REFERENCES ccaa(COD_CCAA),
    CONSTRAINT fk_anio FOREIGN KEY (id_anio)
        REFERENCES anio(id_anio)

);

"""

cursor.execute(query_creacion_demanda)
conexion.commit()

In [17]:
lista_tuplas1 = []
for fila in df_ccaa.values:
    lista_tuplas1.append(tuple(fila))
lista_tuplas1[0]

(4, 'Andalucía')

In [18]:
query_carga1 = 'INSERT INTO ccaa (COD_CCAA, CCAA) VALUES (%s, %s)'

for valores in lista_tuplas1:
    cursor.execute(query_carga1, valores)
conexion.commit()

In [19]:
lista_tuplas2 = []
for fila in df_tipo_generacion.values:
    lista_tuplas2.append(tuple(fila))

query_carga2 = 'INSERT INTO tipo_generacion (id_tipo_generacion, tipo_generacion) VALUES (%s, %s)'

for valores in lista_tuplas2:
    cursor.execute(query_carga2, valores)
conexion.commit()

In [20]:
lista_tuplas3 = []
for fila in df_sexos.values:
    lista_tuplas3.append(tuple(fila))

query_carga3 = 'INSERT INTO sexo (id_sexo, sexo) VALUES (%s, %s)'

for valores in lista_tuplas3:
    cursor.execute(query_carga3, valores)
conexion.commit()

In [21]:
lista_tuplas4 = []
for fila in df_edades.values:
    lista_tuplas4.append(tuple(fila))

query_carga4 = 'INSERT INTO edad (id_grupo_edad, grupo_edad) VALUES (%s, %s)'

for valores in lista_tuplas4:
    cursor.execute(query_carga4, valores)
conexion.commit()

In [22]:
lista_tuplas5 = []
for fila in df_nac_extran.values:
    lista_tuplas5.append(tuple(fila))

query_carga5 = 'INSERT INTO nacionales_extranjeros (id_categoria, categoria) VALUES (%s, %s)'

for valores in lista_tuplas5:
    cursor.execute(query_carga5, valores)
conexion.commit()

In [23]:
lista_tuplas6 = []
for fila in df_provincias.values:
    lista_tuplas6.append(tuple(fila))

query_carga6 = 'INSERT INTO provincias (id_provincia, provincia) VALUES (%s, %s)'

for valores in lista_tuplas6:
    cursor.execute(query_carga6, valores)
conexion.commit()

In [24]:
lista_tuplas7 = []
for fila in df_anio.values:
    lista_tuplas7.append(tuple(fila))
lista_tuplas7 = [tuple(int(valor) for valor in tupla) for tupla in lista_tuplas7]

query_carga7 = 'INSERT INTO anio (id_anio, anio) VALUES (%s, %s)'

for valores in lista_tuplas7:
    cursor.execute(query_carga7, valores)
conexion.commit()

In [25]:
lista_tuplas8 = []
for fila in df_datos_demo.values:
    lista_tuplas8.append(tuple(fila))
lista_tuplas8 = [tuple(int(valor) for valor in tupla) for tupla in lista_tuplas8]

query_carga8 = """INSERT INTO datos_demograficos 
(total, id_provincia, id_grupo_edad, id_categoria, id_sexo, id_anio) 
VALUES (%s, %s, %s, %s, %s, %s)"""

for valores in lista_tuplas8:
    cursor.execute(query_carga8, valores)
conexion.commit()

In [26]:
lista_tuplas9 = []
for fila in df_datos_PIB.values:
    lista_tuplas9.append(tuple(fila))

query_carga9 = """INSERT INTO pib 
(ramas_actividad, total, id_provincia, id_anio) 
VALUES (%s, %s, %s, %s)"""

for valores in lista_tuplas9:
    cursor.execute(query_carga9, valores)
conexion.commit()

In [27]:
lista_tuplas10 = []
for fila in df_datos_generacion.values:
    # Convertir las dos primeras columnas a float y las demás a int
    fila_convertida = (float(fila[0]), float(fila[1]), int(fila[2]), int(fila[3]), int(fila[4]), int(fila[5]))
    lista_tuplas10.append(fila_convertida)

query_carga10 = """INSERT INTO generacion 
(energia_mwh, porcentage_generacion, COD_CCAA, mes, id_tipo_generacion, id_anio) 
VALUES (%s, %s, %s, %s, %s, %s)"""

for valores in lista_tuplas10:
    cursor.execute(query_carga10, valores)
conexion.commit()

In [28]:
lista_tuplas11 = []
for fila in df_demanda.values:
    # Convertir las dos primeras columnas a float y las demás a int
    fila_convertida = (float(fila[0]), int(fila[1]), int(fila[2]), int(fila[3]), int(fila[4]))
    lista_tuplas10.append(fila_convertida)

query_carga11 = """INSERT INTO demanda 
(energia_mwh, porcentage_generacion, COD_CCAA, mes, id_anio) 
VALUES (%s, %s, %s, %s, %s)"""

for valores in lista_tuplas11:
    cursor.execute(query_carga11, valores)
conexion.commit()

In [ ]:
#list(df.itertuples(index=False, name=None)) # para sacar bien los datos cuando vienen como numpy